**Importiere die notwendigen Packages**

In [ ]:
# Empfehlung für Colab: Installiere alles was Du brauchst
# %pip install pandas pivottablejs numpy altair matplotlib geopandas folium

In [4]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [5]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bau_whg_bestand_umbau_umbaukat_ea_zizahlkat_kreis_saldo_jahr_od5102

(int_dwh)

In [6]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "int";
print(status)

int


In [7]:
# Filepath
if status == "prod":
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH\bau_whg_bestand_umbau_umbaukat_ea_zizahlkat_kreis_saldo_jahr_OD5102\BAU510OD5102.csv"
else:
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_whg_bestand_umbau_umbaukat_ea_zizahlkat_kreis_saldo_jahr_OD5102\BAU510OD5102.csv"
print(fp)

\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_whg_bestand_umbau_umbaukat_ea_zizahlkat_kreis_saldo_jahr_OD5102\BAU510OD5102.csv


In [8]:

# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )
print('done')

done


In der Folge ein paar erste Tests:

In [9]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [10]:
data2betested.head(3)

,Jahr,Umbaukategorie,UmbaukategorieCd,Wohnungsgroesse,Wohnungssaldo
0,2015,Aufstockung/Anbau,3,AnzGbdWhgStatZim1_3_saldo,67
1,2015,Aufstockung/Anbau,3,AnzGbdWhgStatZim4plus_saldo,7
2,2015,Innere Reserven,2,AnzGbdWhgStatZim1_3_saldo,6


In [11]:
data2betested.dtypes

Jahr                 int64
Umbaukategorie      object
UmbaukategorieCd     int64
Wohnungsgroesse     object
Wohnungssaldo        int64
dtype: object

In [12]:
data2betested.shape

(70, 5)

Beschreibe einzelne Attribute

In [13]:
data2betested.describe()

,Jahr,UmbaukategorieCd,Wohnungssaldo
count,70,70,70
mean,2018,3,30
std,2,1,61
min,2015,1,-59
25%,2016,2,0
50%,2018,3,6
75%,2020,4,26
max,2021,5,324


Wie viele Nullwerte gibt es im Datensatz?

In [14]:
data2betested.isnull().sum()

Jahr                0
Umbaukategorie      0
UmbaukategorieCd    0
Wohnungsgroesse     0
Wohnungssaldo       0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [15]:
data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested


,Jahr,Umbaukategorie,UmbaukategorieCd,Wohnungsgroesse,Wohnungssaldo
0,2015-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim1_3_saldo,67
1,2015-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim4plus_saldo,7
2,2015-01-01,Innere Reserven,2,AnzGbdWhgStatZim1_3_saldo,6
3,2015-01-01,Innere Reserven,2,AnzGbdWhgStatZim4plus_saldo,-3
4,2015-01-01,Reine Erneuerung,1,AnzGbdWhgStatZim4plus_saldo,24
...,...,...,...,...,...
65,2021-01-01,Reine Erneuerung,1,AnzGbdWhgStatZim1_3_saldo,80
66,2021-01-01,Umnutzung zu Nichtwohnen,5,AnzGbdWhgStatZim4plus_saldo,0
67,2021-01-01,Umnutzung zu Nichtwohnen,5,AnzGbdWhgStatZim1_3_saldo,16
68,2021-01-01,Umnutzung zu Wohnen,4,AnzGbdWhgStatZim4plus_saldo,1


Bereite die Daten für die Grafik unten vor

In [16]:
Jahre = data2betested.groupby("Jahr")
len(Jahre)

7

In [17]:
for key, group in Jahre:
    print(key, len(group))

2015-01-01 00:00:00 10
2016-01-01 00:00:00 10
2017-01-01 00:00:00 10
2018-01-01 00:00:00 10
2019-01-01 00:00:00 10
2020-01-01 00:00:00 10
2021-01-01 00:00:00 10


In [20]:
myJahr = "2021-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr.head(3)


,Jahr,Umbaukategorie,UmbaukategorieCd,Wohnungsgroesse,Wohnungssaldo
60,2021-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim4plus_saldo,16
61,2021-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim1_3_saldo,181
62,2021-01-01,Innere Reserven,2,AnzGbdWhgStatZim1_3_saldo,7


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [21]:
   
test = data2betested.groupby(
   ['Jahr', 'UmbaukategorieCd', 'Umbaukategorie' ]
).agg(
    {
         'Wohnungssaldo':(sum),     # Sum --> (sum)
         #'AnzPers': (sum),   
         #'Wohnungsfl': 'max'  
    }
)
test.head(10)


Wohnungssaldo
Jahr       UmbaukategorieCd Umbaukategorie                         
2015-01-01 1                Reine Erneuerung                    138
           2                Innere Reserven                       3
           3                Aufstockung/Anbau                    74
           4                Umnutzung zu Wohnen                  15
           5                Umnutzung zu Nichtwohnen              0
2016-01-01 1                Reine Erneuerung                    214
           2                Innere Reserven                     -31
           3                Aufstockung/Anbau                    78
           4                Umnutzung zu Wohnen                  55
           5                Umnutzung zu Nichtwohnen             -6

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [22]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [23]:
mySelection = aktuellesJahr[['Jahr', 'UmbaukategorieCd', 'Umbaukategorie', 'Wohnungsgroesse', 'Wohnungssaldo']]
mySelection

,Jahr,UmbaukategorieCd,Umbaukategorie,Wohnungsgroesse,Wohnungssaldo
60,2021-01-01,3,Aufstockung/Anbau,AnzGbdWhgStatZim4plus_saldo,16
61,2021-01-01,3,Aufstockung/Anbau,AnzGbdWhgStatZim1_3_saldo,181
62,2021-01-01,2,Innere Reserven,AnzGbdWhgStatZim1_3_saldo,7
63,2021-01-01,2,Innere Reserven,AnzGbdWhgStatZim4plus_saldo,1
64,2021-01-01,1,Reine Erneuerung,AnzGbdWhgStatZim4plus_saldo,23
65,2021-01-01,1,Reine Erneuerung,AnzGbdWhgStatZim1_3_saldo,80
66,2021-01-01,5,Umnutzung zu Nichtwohnen,AnzGbdWhgStatZim4plus_saldo,0
67,2021-01-01,5,Umnutzung zu Nichtwohnen,AnzGbdWhgStatZim1_3_saldo,16
68,2021-01-01,4,Umnutzung zu Wohnen,AnzGbdWhgStatZim4plus_saldo,1
69,2021-01-01,4,Umnutzung zu Wohnen,AnzGbdWhgStatZim1_3_saldo,7


In [24]:
agg_mySelection = mySelection.groupby(
   ['Jahr', 'UmbaukategorieCd', 'Umbaukategorie', 'Wohnungsgroesse']
).agg(
    {
         'Wohnungssaldo':(sum,min,max)
    }
)
agg_mySelection

Wohnungssaldo  \
                                                                                           sum   
Jahr       UmbaukategorieCd Umbaukategorie           Wohnungsgroesse                             
2021-01-01 1                Reine Erneuerung         AnzGbdWhgStatZim1_3_saldo              80   
                                                     AnzGbdWhgStatZim4plus_saldo            23   
           2                Innere Reserven          AnzGbdWhgStatZim1_3_saldo               7   
                                                     AnzGbdWhgStatZim4plus_saldo             1   
           3                Aufstockung/Anbau        AnzGbdWhgStatZim1_3_saldo             181   
                                                     AnzGbdWhgStatZim4plus_saldo            16   
           4                Umnutzung zu Wohnen      AnzGbdWhgStatZim1_3_saldo               7   
                                                     AnzGbdWhgStatZim4plus_saldo             1   
           5                Umnutzung zu Nichtwohnen AnzGbdWhgStatZim1_3_saldo              16   
                                                     AnzGbdWhgStatZim4plus_saldo             0   

                                                                                            
                                                                                  min  max  
Jahr       UmbaukategorieCd Umbaukategorie           Wohnungsgroesse                        
2021-01-01 1                Reine Erneuerung         AnzGbdWhgStatZim1_3_saldo     80   80  
                                                     AnzGbdWhgStatZim4plus_saldo   23   23  
           2                Innere Reserven          AnzGbdWhgStatZim1_3_saldo      7    7  
                                                     AnzGbdWhgStatZim4plus_saldo    1    1  
           3                Aufstockung/Anbau        AnzGbdWhgStatZim1_3_saldo    181  181  
                                                     AnzGbdWhgStatZim4plus_saldo   16   16  
           4                Umnutzung zu Wohnen      AnzGbdWhgStatZim1_3_saldo      7    7  
                                                     AnzGbdWhgStatZim4plus_saldo    1    1  
           5                Umnutzung zu Nichtwohnen AnzGbdWhgStatZim1_3_saldo     16   16  
                                                     AnzGbdWhgStatZim4plus_saldo    0    0

In [25]:
mySelection = aktuellesJahr[(aktuellesJahr.UmbaukategorieCd != 2) | (aktuellesJahr.Jahr != 1900)]

mySelection.head()

,Jahr,Umbaukategorie,UmbaukategorieCd,Wohnungsgroesse,Wohnungssaldo
60,2021-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim4plus_saldo,16
61,2021-01-01,Aufstockung/Anbau,3,AnzGbdWhgStatZim1_3_saldo,181
62,2021-01-01,Innere Reserven,2,AnzGbdWhgStatZim1_3_saldo,7
63,2021-01-01,Innere Reserven,2,AnzGbdWhgStatZim4plus_saldo,1
64,2021-01-01,Reine Erneuerung,1,AnzGbdWhgStatZim4plus_saldo,23


In [26]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]

highlight = alt.selection(type='single', on='mouseover',
                          fields=['Wohnungsgroesse'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='Umbaukategorie',
    y='Wohnungssaldo',
    color='Wohnungsgroesse'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [27]:
alt.Chart(mySelection).mark_circle(size=60).encode(
    x='Wohnungssaldo',
    y='Umbaukategorie',
    #color='AnzWhg',
    #color=alt.Color('EigentumsartLang', scale=alt.Scale(scheme='dark2')),
    color=alt.Color('Wohnungssaldo', scale=alt.Scale(scheme='greens')),
    tooltip=['Jahr', 'UmbaukategorieCd', 'Umbaukategorie', 'Wohnungsgroesse']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)